# About: Prerequisites for Literate Computing via Notebook
文芸的機械化の準備 - Notebookでね

----

This notebook describes enviromental prerequisits in order to utilize the set of notebooks under this directory.
Prerequisits cover both for the originate host where notebooks operate and for the target hosts manipulated by notebooks.

ここでは本ディレクトリに含まれるNotebookを利用するために必要となる環境条件を明示的に定義している。
環境条件には、Notebookを動作させているホストマシンの環境、およびNotebookで操作しようとしている対象ホスト群の環境が含まれる。

----

自身のNotebook環境にNotebookをコピーした後、読み進めることで条件を満たしているか確認できる。

条件には推奨の度合いによりレベルがあり、 **A=必須、B=推奨、C=任意** というレベルを設定してある。

このNotebookを確認することで、Notebookで操作しようとしている捜査対象ホスト群が **Seedingが済んだ状態である** ことを確認できる。

# Ansibleの設定

Notebook環境は、以下の条件を満たしている必要がある。

## Ansibleがインストールされていること - A

以下のコマンド実行がエラーとならず、インストールされているAnsibleのバージョンが表示されること。

In [1]:
!ansible --version

ansible XXX.XXX.XXX.0
  config file = /etc/ansible/ansible.cfg
  configured module search path = Default w/o overrides


ansibleコマンドがインストールされていない場合は、[Installing the Control Machine](http://docs.ansible.com/ansible/intro_installation.html#installing-the-control-machine)を参考に、Ansibleをインストールする。

## AnsibleでDefault Moduleが利用可能であること - A

DefaultのModuleが使えればよい。特にカスタマイズすべき項目はなし。

# 資材の準備

- Oracle JDK ライセンス認証・RPMパッケージの配置
- インターネットアクセス・Elasticリポジトリに到達可能であること

## JVMの資材が準備されていること - A
Elasticsearchを動作させるのに必要なJVMのファイルを用意します。

Elasticsearchでサポート対象となっているJVMは公式ページの[Support Matrix](https://www.elastic.co/support/matrix#show_jvm)に列挙されております。  
この中でサポート対象のものであればどれでも問題はありませんが、  
本手順ではOracleJVMを利用する前提としております。

Elasticsearch-5.0.0ではバージョン1.8.0_73以降のものが推奨されています。
- [Oracleの公式サイト](http://www.oracle.com/technetwork/java/javase/downloads/java-archive-javase8-2177648.html#jdk-8u112-oth-JPR)からjdk-8uXXXのrpm形式（Notebook作成時はjdk-8u112-linux-x64.rpm）をダウンロードしてください。
- ダウンロードした媒体を /tmp に設置してください。

In [1]:
!ls /tmp/jdk*

/tmp/jdk-8u112-linux-x64.rpm


## Elasticsearchのダウンロードサイトにアクセスできること - A
Elasticsearchの媒体を公式サイトからダウンロード可能か確認します。

次のコマンドを実行し、"200 OK"と表示されれば問題ありません。

In [5]:
!wget --spider -nv --timeout 60 -t 1 https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-5.0.0.rpm

2017-04-06 18:27:16 URL: https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-5.0.0.rpm 200 OK
